In [4]:
import os
from binance.client import Client

In [5]:
API_KEY = os.getenv('BINANCE_API')
API_SECRET = os.getenv('BINANCE_SECRET')

In [7]:
client = Client(api_key = API_KEY, api_secret = API_SECRET, tld = "us")

In [ ]:
client.get_account()